In [1]:
print("ok")

ok


In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains import retrieval_qa
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone as pcn
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import pinecone


In [3]:
PINECONE_API_KEY = "df3c7b7f-791e-43c0-bfbb-05d1423a1cfd"
PINECONE_API_ENV = "us-east-1"


In [4]:
#extract data from the pdf 

def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

### Now we have to convert our corpus to chunks to process into out vector database 

In [6]:
def text_splitter(extracted_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_splitter(extracted_data)
print("number of chunks : ", len(text_chunks))

number of chunks :  7151


## now we have to conver our text chunks into vector

In [8]:
#donloading embedding model 

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [9]:
embedding = download_hugging_face_embedding()

/Users/nishantsaini/Desktop/placement mission /Generative AI/medical chatbot/medbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [11]:
test = embedding.embed_query("hello world")
len(test)
print(test)

[-0.03447728604078293, 0.031023267656564713, 0.006734997034072876, 0.02610897831618786, -0.0393621064722538, -0.16030247509479523, 0.066924087703228, -0.00644137617200613, -0.04745044559240341, 0.014758842997252941, 0.07087539881467819, 0.05552750080823898, 0.019193340092897415, -0.02625138871371746, -0.01010960340499878, -0.026940476149320602, 0.022307490929961205, -0.022226663306355476, -0.149692565202713, -0.01749301701784134, 0.007676353678107262, 0.05435230955481529, 0.003254495793953538, 0.03172589838504791, -0.08462144434452057, -0.0294061116874218, 0.05159566551446915, 0.048124078661203384, -0.0033147772774100304, -0.058279260993003845, 0.04196932166814804, 0.022210663184523582, 0.128188818693161, -0.022338958457112312, -0.011656227521598339, 0.06292832642793655, -0.03287624567747116, -0.09122596681118011, -0.03117532841861248, 0.05269956216216087, 0.04703476279973984, -0.0842030718922615, -0.030056225135922432, -0.020744914188981056, 0.009517857804894447, -0.003721819724887609